In [1]:
import io
import os
import random
from pathlib import Path

In [2]:
%pwd

'c:\\Users\\Asus\\Machine_learning\\LLM\\Projects\\BIRD_benchmark_SQL\\notebook'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Asus\\Machine_learning\\LLM\\Projects\\BIRD_benchmark_SQL'

In [5]:
import os
import json 
from Bird_bench_SQL import logger

In [6]:
from dataclasses import dataclass
from pathlib import Path

In [39]:
@dataclass(frozen=True)
class DataIngestionConfig:
    SQLite_URL      : str
    SQLite_dir      : Path
    SQLite_file     : Path
    data_URL        : str 
    data_dir        : Path
    data_file_path  : Path
    train_file_path : Path
    test_file_path  : Path
    few_shots_path  : Path

In [40]:
from Bird_bench_SQL.constants import *
from Bird_bench_SQL.utils.common import read_yaml,create_directories

In [44]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath    = CONFIG_FILE_PATH,
                 params_filepath    = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:

        config      = self.config.data_ingestion
        create_directories([config.SQLite_dir,config.data_dir])

        data_ingestion_config= DataIngestionConfig(SQLite_URL       = config.SQLite_URL,
                                                   SQLite_dir       = config.SQLite_dir,
                                                   SQLite_file      = config.SQLite_file,
                                                   data_URL         = config.data_URL,
                                                   data_dir         = config.data_dir,
                                                   data_file_path   = config.data_file_path,
                                                   train_file_path  = config.train_file_path,
                                                   test_file_path   = config.test_file_path,
                                                   few_shots_path   = config.few_shots_path)
        return data_ingestion_config

In [11]:
from  urllib import request 
import zipfile

In [45]:
manager = ConfigurationManager()
manager_config = manager.get_data_ingestion_config()

[2024-09-07 23:35:28,480: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-07 23:35:28,480: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-07 23:35:28,480: INFO: common: created directory at: artifacts]
[2024-09-07 23:35:28,480: INFO: common: created directory at: artifacts/SQLite_database]
[2024-09-07 23:35:28,480: INFO: common: created directory at: artifacts/data]


In [46]:
manager_config.data_file_path

'artifacts/data/train.json'

In [43]:
class DataIngestion:

    def __init__(self,config = DataIngestionConfig):
        self.config = config
        
    def download_file(self):

        if not os.path.exists(self.config.SQLite_file):
            logger.info(f"Downloading SQLite file from {self.config.SQLite_URL} to {self.config.SQLite_dir}")
            filename, header = request.urlretrieve(url      = self.config.SQLite_URL, 
                                                   filename = self.config.SQLite_file)
            logger.info(f"{filename} downloaded with the following info: \n{header}")
        else:
            logger.info(f"{self.config.SQLite_file} already exists, skipping download.")

        # Download data file if it doesn't exist
        if not os.path.exists(self.config.data_file_path):
            logger.info(f"Downloading data file from {self.config.data_URL} to {self.config.data_dir}")
            filename, header = request.urlretrieve(url      = self.config.data_URL,
                                                   filename = self.config.data_file_path)
            logger.info(f"{filename} downloaded with the following info: \n{header}")
        else:
            logger.info(f"{self.config.data_dir} already exists, skipping download.")

In [47]:
try:
    config                  = ConfigurationManager()
    data_ingestion_config   = config.get_data_ingestion_config()
    data_ingestion          = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()

except Exception as e:
    raise e

[2024-09-07 23:36:32,550: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-07 23:36:32,550: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-07 23:36:32,550: INFO: common: created directory at: artifacts]
[2024-09-07 23:36:32,559: INFO: common: created directory at: artifacts/SQLite_database]
[2024-09-07 23:36:32,560: INFO: common: created directory at: artifacts/data]
[2024-09-07 23:36:32,560: INFO: 3777581137: Downloading SQLite file from https://github.com/Vampaxx/BIRD_benchmark_SQL/blob/main/artifacts/movies_4.sqlite to artifacts/SQLite_database]
[2024-09-07 23:36:33,440: INFO: 3777581137: artifacts/SQLite_database/movies_4.sqlite downloaded with the following info: 
Server: GitHub.com
Date: Sat, 07 Sep 2024 18:06:32 GMT
Content-Type: text/html; charset=utf-8
Vary: X-PJAX, X-PJAX-Container, Turbo-Visit, Turbo-Frame, Accept-Encoding, Accept, X-Requested-With
ETag: W/"3343089f93c6d5f6059c0ccb579c7cec"
Cache-Control: max-age=0, private, must-